### Split the data on the disk into a train, validation and test set

1. Create for each tile on folder which contains the folder from every season for this tile
2. Random sample tiles for the train, validation and test set
3. Copy the tiles from the disk to the train, validation and test folder

In [1]:
from pathlib import Path
import rasterio
import re
import os
import random
import shutil
import torch
from tqdm import tqdm

In [4]:
IDENTIFIER_REGEX = re.compile(
    r"""(?P<mission>S2[A-B])_MSI
        (?P<product_level>L[1-2][A-C])_
        (?P<sensing_time>\d{8}T\d{6})_
        (?P<processing_baseline>N\d{4})_
        (?P<relative_orbit>R\d{3})_T
        (?P<utm_code>\d{2})
        (?P<latitude_band>\w{1})
        (?P<square>\w{2})_
        (?P<year>\d{4})
        (?P<month>\d{2})
        (?P<day>\d{2})T
        (?P<product_time>\d{6})""",
    re.VERBOSE,
)

In [6]:
def get_tile_and_date(identifier: str):
    regex_match = re.search(IDENTIFIER_REGEX, identifier)

    if not regex_match:
        return None, None

    utm_code = regex_match.group("utm_code")
    latitude_band = regex_match.group("latitude_band")
    square = regex_match.group("square")
    year = regex_match.group("year")
    # remove leading zeros
    month = str(int(regex_match.group("month")))
    day = str(int(regex_match.group("day")))

    tile = f"{utm_code}{latitude_band}{square}"
    tile_date = f"{year}-{month}-{day}"

    return tile, tile_date

In [47]:
images_prepro_path = Path(r'C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\data_local\images_only_AOI_test_color_corr_cleaned')
masks_prepro_path = Path(r'C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\data_local\masks_only_AOI_test_color_corr_cleaned')

## Create a list of all tiles which are available on the disk

In [ ]:
def index_to_filename(image_dir: Path, set_list: list) -> list:
    set_filenames = []
    for file_path in image_dir.glob("*.pt"):
        tile, number, date = file_path.stem.split("_")
        if (tile, number) in set_list:
            set_filenames.append(str(file_path))  # .name
    return set_filenames

In [52]:
# create a list of all tiles
tile_list = []
for file_path in images_prepro_path.glob("*.pt"):
    filename = file_path.stem
    tile, number, date = filename.split("_")
    tile_list.append(tile)

# set is used to get unique values of tiles
tiles_unique = list(set(tile_list))

In [ ]:
# select random tiles for train, val and test
random.seed(42)
random.shuffle(tiles_unique)

num_total = len(tiles_unique)
num_train = int(num_total * 0.7)
num_val = int(num_total * 0.1)

In [54]:
# create train, val and test list
train_list = tiles_unique[:num_train]
val_list = tiles_unique[num_train : num_train + num_val]
test_list = tiles_unique[num_train + num_val :]

In [56]:
# create a directory for each set
train_dir = Path(r'C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\data_local\data_splitted')
for set_name in ["train", "val", "test"]:
    os.makedirs(train_dir / set_name, exist_ok=True)
    os.makedirs(train_dir / set_name / "images", exist_ok=True)
    os.makedirs(train_dir / set_name / "masks", exist_ok=True)

In [69]:
for file in images_prepro_path.glob("*.pt"):
    file_stammed = file.stem
    filename = file.name
    tile, _, _ = file_stammed.split("_")
    if tile in train_list:
        shutil.copyfile(images_prepro_path / filename, train_dir / "train" / "images" / filename)
        shutil.copyfile(masks_prepro_path / filename, train_dir / "train" / "masks" / filename)
    elif tile in val_list:
        shutil.copyfile(images_prepro_path / filename, train_dir / "val" / "images" / filename)
        shutil.copyfile(masks_prepro_path / filename, train_dir / "val" / "masks" / filename)
    elif tile in test_list:
        shutil.copyfile(images_prepro_path / filename, train_dir / "test" / "images" / filename)
        shutil.copyfile(masks_prepro_path / filename, train_dir / "test" / "masks" / filename)
    else:
        print("Error: Tile not in any set!")

In [73]:
# check if all files are copied
print("Number of files in each set:")
print(f'Train images: {len(list(train_dir.glob("train/images/*.pt")))}')
print(f'Train masks: {len(list(train_dir.glob("train/masks/*.pt")))}')
print(f'Val images: {len(list(train_dir.glob("val/images/*.pt")))}')
print(f'Val masks: {len(list(train_dir.glob("val/masks/*.pt")))}')
print(f'Test images: {len(list(train_dir.glob("test/images/*.pt")))}')
print(f'Test masks: {len(list(train_dir.glob("test/masks/*.pt")))}')

Number of files in each set:
Train images: 3832
Train masks: 3832
Val images: 489
Val masks: 489
Test images: 342
Test masks: 342


In [ ]:
print("Sum of all files:")
print(f'Images: {len(list(train_dir.glob("**/*.pt")))}')

### Same as before, but now with the undersampled data

Additionally the data needs to be transformed from .tiff to tensor (.pt) format

In [6]:
images_prepro_path = Path(r'C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\data_local\images_undersampling')
masks_prepro_path = Path(r'C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\data_local\masks_undersampling')

In [7]:
# create a list of all tiles
tile_list = []
for file_path in images_prepro_path.glob("*.tif"):
    filename = file_path.stem
    tile, number, date = filename.split("_")
    tile_list.append(tile)

# set is used to get unique values of tiles
tiles_unique = list(set(tile_list))

In [36]:
print(f'Number of tiles: {len(tiles_unique)}')

Number of tiles: 56


In [44]:
# select random tiles for train, val and test
random.seed(42)
random.shuffle(tiles_unique)

num_total = len(tiles_unique)
num_train = int(num_total * 0.7)
num_val = int(num_total * 0.2)

In [45]:
# create train, val and test list
train_list = tiles_unique[:num_train]
val_list = tiles_unique[num_train : num_train + num_val]
test_list = tiles_unique[num_train + num_val :]

In [46]:
print("Number of tiles in each set:")
print(f'Train: {len(train_list)}')
print(f'Val: {len(val_list)}')
print(f'Test: {len(test_list)}')

Number of tiles in each set:
Train: 39
Val: 11
Test: 6


In [41]:
# create a directory for each set
data_dir = Path(r'C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\data_local\data_splitted_undersampling')
for set_name in ["train", "val", "test"]:
    os.makedirs(data_dir / set_name, exist_ok=True)
    os.makedirs(data_dir / set_name / "images", exist_ok=True)
    os.makedirs(data_dir / set_name / "masks", exist_ok=True)

In [42]:
num_files = len(list(images_prepro_path.glob("*.tif")))
for file in tqdm(images_prepro_path.glob("*.tif"), total=num_files):
    file_stammed = file.stem
    filename = file.name
    tile, _, _ = file_stammed.split("_")

    image_path = images_prepro_path / filename
    mask_path = masks_prepro_path / filename
    
    image = rasterio.open(image_path).read()
    mask = rasterio.open(mask_path).read()
    
    image_tensor = torch.from_numpy(image)
    mask_tensor = torch.from_numpy(mask)

    if tile in train_list:
        torch.save(image_tensor, data_dir / "train" / "images" / f"{file_stammed}.pt")
        torch.save(mask_tensor, data_dir / "train" / "masks" / f"{file_stammed}.pt")
    elif tile in val_list:
        torch.save(image_tensor, data_dir / "val" / "images" / f"{file_stammed}.pt")
        torch.save(mask_tensor, data_dir / "val" / "masks" / f"{file_stammed}.pt")
    elif tile in test_list:
        torch.save(image_tensor, data_dir / "test" / "images" / f"{file_stammed}.pt")
        torch.save(mask_tensor, data_dir / "test" / "masks" / f"{file_stammed}.pt")
    else:
        print("Error: Tile not in any set!")

  0%|          | 0/20683 [00:00<?, ?it/s]

100%|██████████| 20683/20683 [09:50<00:00, 35.03it/s]


In [47]:
# check if all files are copied
print("Number of files in each set:")
print(f'Train images: {len(list(data_dir.glob("train/images/*.pt")))}')
print(f'Train masks: {len(list(data_dir.glob("train/masks/*.pt")))}')
print(f'Val images: {len(list(data_dir.glob("val/images/*.pt")))}')
print(f'Val masks: {len(list(data_dir.glob("val/masks/*.pt")))}')
print(f'Test images: {len(list(data_dir.glob("test/images/*.pt")))}')
print(f'Test masks: {len(list(data_dir.glob("test/masks/*.pt")))}')

Number of files in each set:
Train images: 14178
Train masks: 14178
Val images: 4648
Val masks: 4648
Test images: 1857
Test masks: 1857


### Create geojson file 

In [79]:
import geopandas as gpd
import rasterio.features
import numpy as np
from typing import Tuple, List
from shapely.geometry import Polygon
from rasterio.warp import transform_geom
from datetime import datetime


In [4]:
# strucutre of geojson
gdf = gpd.read_file(r'C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\data_local\geojsons\solar-parks.geojson')
gdf.head()

,avg_confidence,comment,date_of_data,first_detection,is_valid,last_detection,name_in_aws,name_of_model,peak_power,size_in_sq_m,geometry
0,0.0,None,2023-05-09,2023-05-09,non-valid,2023-05-09,32UPC_20_2023-5-9.tif,solar-park-detection,3.088,19300.0,"POLYGON ((11.20714 52.31969, 11.20787 52.31968..."
1,0.0,None,2023-05-09,2023-05-09,non-valid,2023-05-09,32UPC_20_2023-5-9.tif,solar-park-detection,4.512,28200.0,"POLYGON ((11.20472 52.31821, 11.20530 52.31820..."
2,0.0,None,2023-05-09,2023-05-09,valid,2023-05-09,32UPC_31_2023-5-9.tif,solar-park-detection,2.384,14900.0,"POLYGON ((11.61811 52.32361, 11.61870 52.32360..."
3,0.0,None,2023-05-09,2023-05-09,valid,2023-05-09,32UPC_33_2023-5-9.tif,solar-park-detection,4.672,29200.0,"POLYGON ((11.68509 52.30520, 11.68568 52.30519..."
4,0.0,None,2023-05-09,2023-05-09,non-valid,2023-05-09,32UPC_193_2023-5-9.tif,solar-park-detection,1.408,8800.0,"POLYGON ((11.09063 52.23873, 11.09060 52.23810..."


In [19]:
data_dir = Path(r'C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\data_local\data_splitted_undersampling')
mask_dir = Path(r'C:\Users\Fabian\Documents\Github_Masterthesis\Solarpark-detection\data_local\masks_undersampling')
file_list = data_dir.glob("test/masks/*.pt")

In [75]:
def masks_to_polygons(
    masks: np.ndarray, metadata: dict
) -> Tuple[List[Polygon], List[float]]:
    masks = masks.astype(np.uint8)
    transform = metadata["transform"]
    crs = metadata["crs"]
    # extract shapes
    shapes = rasterio.features.shapes(masks, transform=transform)
    polygons = []
    areas = []
    peak_powers = []
    for shape in shapes:
        if shape[1] == 1:
            polygon = Polygon(shape[0]["coordinates"][0])
            area = polygon.area
            # realistically, the smallest solar park should be bigger than 10.000 m²
            if area >= 0:
                # Transform the polygon coordinates to EPSG:4326
                transformed_geom = transform_geom(
                    crs, "EPSG:4326", polygon.__geo_interface__
                )
                transformed_coords = transformed_geom["coordinates"][0]
                transformed_polygon = Polygon(transformed_coords)
                peak_power = calc_peak_power(area_in_sq_m=area)
                polygons.append(transformed_polygon)
                areas.append(area)
                peak_powers.append(peak_power)
    return polygons, areas, peak_powers

In [72]:
def calc_peak_power(area_in_sq_m: float) -> float:
    # https://www.bundesnetzagentur.de/SharedDocs/Downloads/DE/Sachgebiete/Energie/Unternehmen_Institutionen/ErneuerbareEnergien/PV-Freiflaechenanlagen/Bericht_Flaecheninanspruchnahme_2016.pdf?__blob=publicationFile&v=2#:~:text=Die%20bereits%20im%20Rahmen%20der,Ackerland%20in%20benachteiligten%20Gebieten%20errichtet. # noqa
    # page 8 on the pdf (german)
    # 1,6 acre = 1 MWp
    # area in sq m / 10000 = area in acr * 1.6 = peak power in MWp
    return area_in_sq_m / 10000 * 1.6

In [90]:
polygons_list = []
areas_list = []
peak_powers_list = []
filenames_list = []
for file_path in file_list:
    filename = file_path.stem
    mask = rasterio.open(mask_dir / f"{filename}.tif").read()
    metadata   = rasterio.open(mask_dir / f"{filename}.tif").meta
    if mask.sum() > 0:
        polygons, areas, peak_powers = masks_to_polygons(mask, metadata)
        polygons_list.extend(polygons)
        areas_list.extend(areas)
        peak_powers_list.extend(peak_powers)
        for i in range(len(polygons)):
            filenames_list.append(f"{filename}.tif")

In [77]:
gdf.columns

Index(['avg_confidence', 'comment', 'date_of_data', 'first_detection',
       'is_valid', 'last_detection', 'name_in_aws', 'name_of_model',
       'peak_power', 'size_in_sq_m', 'geometry'],
      dtype='object')

In [97]:
date_string = "2018-01-01"
date_obj = datetime.strptime(date_string, "%Y-%m-%d").date()
data = {
    "name_of_model": "test",
    "size_in_sq_m": areas_list,
    "peak_power": peak_powers_list,
    "date_of_data": '2018-01-01',
    "first_detection": '2018-01-01',  # will be handled on api level
    "last_detection": '2018-01-01',  # will be handled on api level
    "avg_confidence": 0,
    "name_in_aws": filenames_list,
    "is_valid": "None",
    "comment": "test",
    "geometry": polygons_list
}
df = gpd.GeoDataFrame(data)

In [98]:
df

,name_of_model,size_in_sq_m,peak_power,date_of_data,first_detection,last_detection,avg_confidence,name_in_aws,is_valid,comment,geometry
0,test,29200.0,4.672,2018-01-01,2018-01-01,2018-01-01,0,32UMA_1034_2018-11-16.tif,None,test,"POLYGON ((8.38072 50.02334, 8.38617 50.02337, ..."
1,test,29200.0,4.672,2018-01-01,2018-01-01,2018-01-01,0,32UMA_1034_2018-4-20.tif,None,test,"POLYGON ((8.38072 50.02334, 8.38617 50.02337, ..."
2,test,29200.0,4.672,2018-01-01,2018-01-01,2018-01-01,0,32UMA_1034_2018-9-27.tif,None,test,"POLYGON ((8.38072 50.02334, 8.38617 50.02337, ..."
3,test,38700.0,6.192,2018-01-01,2018-01-01,2018-01-01,0,32UMA_1042_2018-11-16.tif,None,test,"POLYGON ((8.64275 50.02427, 8.64303 50.02427, ..."
4,test,97900.0,15.664,2018-01-01,2018-01-01,2018-01-01,0,32UMA_1042_2018-11-16.tif,None,test,"POLYGON ((8.63968 50.02444, 8.64163 50.02444, ..."
...,...,...,...,...,...,...,...,...,...,...,...
625,test,41600.0,6.656,2018-01-01,2018-01-01,2018-01-01,0,32UPF_1622_2018-4-20.tif,None,test,"POLYGON ((12.01673 54.17422, 12.01780 54.17419..."
626,test,41600.0,6.656,2018-01-01,2018-01-01,2018-01-01,0,32UPF_1622_2018-7-6.tif,None,test,"POLYGON ((12.01673 54.17422, 12.01780 54.17419..."
627,test,83400.0,13.344,2018-01-01,2018-01-01,2018-01-01,0,32UPF_1623_2018-10-14.tif,None,test,"POLYGON ((12.01887 54.17417, 12.01869 54.17165..."
628,test,83400.0,13.344,2018-01-01,2018-01-01,2018-01-01,0,32UPF_1623_2018-4-20.tif,None,test,"POLYGON ((12.01887 54.17417, 12.01869 54.17165..."


In [99]:
df = df.set_geometry('geometry')

df.to_file('testset.geojson', driver='GeoJSON')